### Import dependencies

In [1]:
import pandas as pd

In [2]:
#view settings
pd.set_option('display.max_column', None)
# pd.set_option('display.max_row', None)
pd.set_option('display.max_colwidth', None)

#reset
# pd.reset_option('all')

### Load dataset

In [3]:
loan_df = pd.read_csv('./dataset/ppp_over_150k.csv')
fraud_df = pd.read_csv('./dataset/ppp_fraud_cases.csv')
naics_df = pd.read_csv('./dataset/naics_code.csv', delimiter='\t')
sub_df = pd.read_csv('./dataset/subsector.csv',delimiter='-')

In [4]:
# loan_df = loan_df.head(100)
sub_df = pd.read_csv('./dataset/subsector.csv',delimiter='-')

### Drop unnecessary fields

In [5]:
loan_df.keys()

Index(['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'ProcessingMethod',
       'BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'OriginatingLender', 'Originatin

In [6]:
to_remove = ['BorrowerAddress', 'LoanStatusDate',
       'SBAGuarantyPercentage', 'UndisbursedAmount',
       'ServicingLenderLocationID',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'OriginatingLenderLocationID',
       'OriginatingLender', 'OriginatingLenderCity', 'OriginatingLenderState',
       'NonProfit', 'ForgivenessAmount',
       'ForgivenessDate', 'ApprovalDiff', 'NotForgivenAmount',
       'ForgivenPercentage']

In [7]:
loan_df = loan_df.drop(columns=to_remove)

### Change to datetime format

In [8]:
loan_df['DateApproved'] = pd.to_datetime(loan_df['DateApproved'])

### isAfterBiden

In [9]:
loan_df['isAfterBiden'] = 0
test = loan_df['DateApproved'] >= "2021-01-21"
for i in range(0, loan_df.shape[0]):
    if test[i] == True:
        loan_df.at[i, 'isAfterBiden'] = 1

### isDiffBorrowerAndProjectCity

In [11]:
def levenshtein_distance(word1, word2):
    # Create a matrix to store the edit distances
    matrix = [[0] * (len(word2) + 1) for _ in range(len(word1) + 1)]

    # Initialize the matrix
    for i in range(len(word1) + 1):
        matrix[i][0] = i
    for j in range(len(word2) + 1):
        matrix[0][j] = j

    # Fill in the matrix with edit distances
    for i in range(1, len(word1) + 1):
        for j in range(1, len(word2) + 1):
            cost = 0 if word1[i - 1] == word2[j - 1] else 1
            matrix[i][j] = min(
                matrix[i - 1][j] + 1,  # Deletion
                matrix[i][j - 1] + 1,  # Insertion
                matrix[i - 1][j - 1] + cost,  # Substitution
            )

    # Calculate the Levenshtein distance
    levenshtein_distance = matrix[len(word1)][len(word2)]

    # Calculate the percentage similarity
    max_length = max(len(word1), len(word2))
    similarity_percentage = (1 - (levenshtein_distance / max_length)) * 100

    return similarity_percentage

In [12]:
loan_df['isDiffBorrowerAndProjectCity'] = 1

for i in range (0,loan_df.shape[0]):
    city1 = loan_df.at[i, 'BorrowerCity']
    city2 = loan_df.at[i, 'ProjectCity']
    # print(city1, city2)
    similarity = levenshtein_distance(city1, city2)
    if city1 == city2:
        loan_df.at[i, 'isDiffBorrowerAndProjectCity'] = 0
    if similarity >= 80:
        loan_df.at[i, 'isDiffBorrowerAndProjectCity'] = 0

### Get Industry from NAICS

In [13]:
# Convert the DataFrame to a dictionary
naics_dict = dict(zip(naics_df['Code'], naics_df['Category']))

In [14]:
loan_df['Industry'] = ''

for i in range (0, loan_df.shape[0]):
    naics_str = str(loan_df.at[i, 'NAICSCode'])
    code = int(naics_str[:2])
    loan_df.at[i, 'Industry'] = naics_dict[code]

### Get subsector

In [15]:
sub_dict = dict(zip(sub_df['Code'], sub_df['Subsector']))

In [16]:
loan_df = loan_df[loan_df['NAICSCode'].apply(lambda x: len(str(x)) >= 3)]
loan_df = loan_df.reset_index(drop=['index'])


In [17]:
loan_df['Subsector'] = ''
missing = []
for i in range (0, loan_df.shape[0]):
    naics_str = str(loan_df.at[i, 'NAICSCode'])
    code = int(naics_str[:3])
    # try:
    loan_df.at[i, 'Subsector'] = sub_dict[code]
    # except KeyError:
    #     missing += [code]

### Get dummy from categorical

In [18]:
to_dummy = ['ProcessingMethod',
       'LoanStatus','RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'Race', 'Ethnicity', 'BusinessType', 
       'Gender', 'Veteran', 'Industry']

In [19]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

for field in to_dummy:
    loan_df[field+'_encoded'] = label_encoder.fit_transform(loan_df[field])


### Scale numerical data that is too big (drop all inf/none val)

In [20]:
to_scale = ['InitialApprovalAmount', 'CurrentApprovalAmount',
       'TOTAL_PROCEED',
       'PROCEED_Per_Job']

In [21]:
# Replace infinite values with None
for field in to_scale:
    loan_df[field] = loan_df[field].replace([float('inf'), -float('inf')], None)

In [22]:
loan_df[to_scale].isnull().sum()

InitialApprovalAmount    0
CurrentApprovalAmount    0
TOTAL_PROCEED            0
PROCEED_Per_Job          4
dtype: int64

In [23]:
#remove row where Proceed_Per_Job is null
loan_df = loan_df[loan_df['PROCEED_Per_Job'].isnull() == False]
loan_df = loan_df.reset_index(drop=['index'])

In [24]:
from sklearn.preprocessing import MinMaxScaler  # StandardScaler

scaler = MinMaxScaler()

# Scale the columns and create new scaled columns
for field in to_scale:
    loan_df[field+'_scaled'] = scaler.fit_transform(loan_df[[field]])

### Standardise numerical data

In [25]:
to_standardise = ['Term', 'JobsReported','PROCEED_Diff', 'UTILITIES_PROCEED_pct',
       'PAYROLL_PROCEED_pct', 'MORTGAGE_INTEREST_PROCEED_pct',
       'RENT_PROCEED_pct', 'REFINANCE_EIDL_PROCEED_pct',
       'HEALTH_CARE_PROCEED_pct', 'DEBT_INTEREST_PROCEED_pct',
       'InitialApprovalAmount_scaled', 'CurrentApprovalAmount_scaled',
       'TOTAL_PROCEED_scaled', 'PROCEED_Per_Job_scaled']

In [26]:
from sklearn.preprocessing import StandardScaler  # StandardScaler

scaler = StandardScaler()

# Scale the columns and create new scaled columns
for field in to_standardise:
    loan_df[field+'_standardised'] = scaler.fit_transform(loan_df[[field]])

### Join with fraud dataset

In [27]:
df = pd.merge(loan_df, fraud_df, left_on=['BorrowerName', 'BorrowerState'], right_on=['Company', 'State'], how='left')

In [28]:
df = df.fillna('')

In [29]:
df['isFraud'] = 0
for i in range (0, df.shape[0]):
    company = df.at[i, 'Company']
    if company != '':
        df.at[i, 'isFraud'] = 1

### Drop unnecessary field

In [30]:
df = df.drop(columns=['Company', 'State', 'Source', 'Suspects','InitialApprovalAmount_scaled','CurrentApprovalAmount_scaled','TOTAL_PROCEED_scaled','PROCEED_Per_Job_scaled'])

In [31]:
df.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerCity,BorrowerState,BorrowerZip,LoanStatus,Term,InitialApprovalAmount,CurrentApprovalAmount,FranchiseName,ServicingLenderName,ServicingLenderZip,RuralUrbanIndicator,HubzoneIndicator,LMIIndicator,BusinessAgeDescription,ProjectCity,JobsReported,NAICSCode,Race,Ethnicity,BusinessType,Gender,Veteran,TOTAL_PROCEED,PROCEED_Diff,UTILITIES_PROCEED_pct,PAYROLL_PROCEED_pct,MORTGAGE_INTEREST_PROCEED_pct,RENT_PROCEED_pct,REFINANCE_EIDL_PROCEED_pct,HEALTH_CARE_PROCEED_pct,DEBT_INTEREST_PROCEED_pct,PROCEED_Per_Job,isAfterBiden,isDiffBorrowerAndProjectCity,Industry,Subsector,ProcessingMethod_encoded,LoanStatus_encoded,RuralUrbanIndicator_encoded,HubzoneIndicator_encoded,LMIIndicator_encoded,BusinessAgeDescription_encoded,Race_encoded,Ethnicity_encoded,BusinessType_encoded,Gender_encoded,Veteran_encoded,Industry_encoded,Term_standardised,JobsReported_standardised,PROCEED_Diff_standardised,UTILITIES_PROCEED_pct_standardised,PAYROLL_PROCEED_pct_standardised,MORTGAGE_INTEREST_PROCEED_pct_standardised,RENT_PROCEED_pct_standardised,REFINANCE_EIDL_PROCEED_pct_standardised,HEALTH_CARE_PROCEED_pct_standardised,DEBT_INTEREST_PROCEED_pct_standardised,InitialApprovalAmount_scaled_standardised,CurrentApprovalAmount_scaled_standardised,TOTAL_PROCEED_scaled_standardised,PROCEED_Per_Job_scaled_standardised,isFraud
0,9547507704,2020-05-01,464,PPP,sumter coatings inc,sumter,UNK,29150-9662,Paid in Full,24,769358.78,769358.78,NonFranchise,synovus bank,31901-2429,U,N,N,Existing or more than 2 years old,sumter,62.0,325510,Unanswered,Unknown/NotStated,Corporation,Unanswered,Unanswered,769358.78,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12409.01,0,0,Manufacturing,Chemical Manufacturing,0,2,1,0,0,1,7,2,5,2,1,10,-0.718289,0.151037,-0.004199,-0.191335,0.332254,-0.117829,-0.259507,-0.024109,-0.175324,-0.069127,0.316451,0.321404,0.321407,-0.004019,0
1,9777677704,2020-05-01,464,PPP,pleasant places inc,north charleston,UNK,29420-9000,Paid in Full,24,736927.79,736927.79,NonFranchise,synovus bank,31901-2429,U,Y,Y,Existing or more than 2 years old,north charleston,73.0,561730,White,Unknown/NotStated,Sole Proprietorship,Male Owned,Non-Veteran,736927.79,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,10094.90,0,0,Administrative and support and waste management and remediation services,Administrative and Support Services,0,2,1,1,1,1,8,2,20,1,0,1,-0.718289,0.314426,-0.004199,-0.191335,0.332254,-0.117829,-0.259507,-0.024109,-0.175324,-0.069127,0.272954,0.277532,0.277535,-0.118508,0
2,6223567700,2020-05-01,920,PPP,kirtley construction inc,san bernardino,UNK,92407-1740,Paid in Full,24,499871.00,499871.00,NonFranchise,bank of america national association,28202-4024,U,N,N,New Business or 2 years or less,san bernardino,21.0,236115,American Indian or Alaska Native,Not Hispanic or Latino,Corporation,Male Owned,Non-Veteran,499871.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,23803.38,0,0,Construction,Construction of Buildings,0,2,1,0,0,2,0,1,5,1,0,4,-0.718289,-0.457959,-0.004199,-0.191335,0.332254,-0.117829,-0.259507,-0.024109,-0.175324,-0.069127,-0.044994,-0.043156,-0.043153,0.559708,0
3,9662437702,2020-05-01,101,PPP,aero box llc,unknown,UNK,0,Paid in Full,24,367437.00,367437.00,NonFranchise,the huntington national bank,43215-3413,U,N,N,Unanswered,unknown,25.0,484210,Unanswered,Unknown/NotStated,Unknown,Unanswered,Unanswered,367437.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,14697.48,0,0,Transportation and warehousing,Truck Transportation,0,2,1,0,0,4,7,2,25,2,1,18,-0.718289,-0.398545,-0.004199,-0.191335,0.332254,-0.117829,-0.259507,-0.024109,-0.175324,-0.069127,-0.222618,-0.222312,-0.222309,0.109201,0
4,9774337701,2020-05-01,101,PPP,hudson extrusions inc,unknown,UNK,0,Paid in Full,24,328840.00,328840.00,NonFranchise,the huntington national bank,43215-3413,U,N,N,Unanswered,unknown,22.0,326199,Unanswered,Unknown/NotStated,Unknown,Unanswered,Unanswered,328840.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,14947.27,0,0,Manufacturing,Plastics and Rubber Products Manufacturin

### Write to CSV

In [34]:
df.to_csv('./dataset/ppp_transformed.csv', index=False)